# **THIS WORKBOOK GENERATED IN GOOGLE COLAB (https://colab.research.google.com/)**
## **READ CSV FILES INTO PANDAS DATA FRAME, CONVERT TO SQLITE AND QUERY**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3

## Pandas ==> SQL helper functions from https://gist.githubusercontent.com/stephenleo
https://towardsdatascience.com/have-a-sql-interview-coming-up-ace-it-using-google-colab-6d3c0ffb29dc

In [ ]:
#@title pd_to_sqlDB Helper Function
def pd_to_sqlDB(input_df: pd.DataFrame,
                table_name: str,
                db_name: str = 'default.db') -> None:

    '''Take a Pandas dataframe `input_df` and upload it to `table_name` SQLITE table
    Args:
        input_df (pd.DataFrame): Dataframe containing data to upload to SQLITE
        table_name (str): Name of the SQLITE table to upload to
        db_name (str, optional): Name of the SQLITE Database in which the table is created. 
                                 Defaults to 'default.db'.
    '''

    # Step 1: Setup local logging
    import logging
    logging.basicConfig(level=logging.INFO,
                        format='%(asctime)s %(levelname)s: %(message)s',
                        datefmt='%Y-%m-%d %H:%M:%S')

    # Step 2: Find columns in the dataframe
    cols = input_df.columns
    cols_string = ','.join(cols)
    val_wildcard_string = ','.join(['?'] * len(cols))

    # Step 3: Connect to a DB file if it exists, else crete a new file
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    logging.info(f'SQL DB {db_name} created')

    # Step 4: Create Table
    sql_string = f"""CREATE TABLE {table_name} ({cols_string});"""
    cur.execute(sql_string)
    logging.info(f'SQL Table {table_name} created with {len(cols)} columns')

    # Step 5: Upload the dataframe
    rows_to_upload = input_df.to_dict(orient='split')['data']
    sql_string = f"""INSERT INTO {table_name} ({cols_string}) VALUES ({val_wildcard_string});"""    
    cur.executemany(sql_string, rows_to_upload)
    logging.info(f'{len(rows_to_upload)} rows uploaded to {table_name}')
  
    # Step 6: Commit the changes and close the connection
    con.commit()
    con.close()

In [ ]:
#@title sql_query_to_pd Helper Function
def sql_query_to_pd(sql_query_string: str, db_name: str ='default.db') -> pd.DataFrame:
    '''Execute an SQL query and return the results as a pandas dataframe
    Args:
        sql_query_string (str): SQL query string to execute
        db_name (str, optional): Name of the SQLITE Database to execute the query in.
                                 Defaults to 'default.db'.
    Returns:
        pd.DataFrame: Results of the SQL query in a pandas dataframe
    '''    
    # Step 1: Connect to the SQL DB
    con = sqlite3.connect(db_name)

    # Step 2: Execute the SQL query
    cursor = con.execute(sql_query_string)

    # Step 3: Fetch the data and column names
    result_data = cursor.fetchall()
    cols = [description[0] for description in cursor.description]

    # Step 4: Close the connection
    con.close()

    # Step 5: Return as a dataframe
    return pd.DataFrame(result_data, columns=cols)

# Connect to Data on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
file_location = '/content/drive/MyDrive/Data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Step 1: Read the csv files into dataframes
## Datasets from https://data.cityofchicago.org/, using small subset of data

In [ ]:
crime_df = pd.read_csv('/content/drive/MyDrive/Data/Chicago/ChicagoCrimeData.csv')
census_df = pd.read_csv('/content/drive/MyDrive/Data/Chicago/ChicagoCensusData.csv')
schools_df = pd.read_csv('/content/drive/MyDrive/Data/Chicago/ChicagoPublicSchools.csv')

In [ ]:
#@title Clean up Column Names in Schools Table to Avoid SQLite Errors

#schools_df.drop(list(schools_df.filter(regex = '9th ')), axis = 1, inplace = True)
schools_df.rename(columns=lambda s: s.replace("9th ", "Ninth"), inplace=True)
schools_df.rename(columns=lambda s: s.replace("10th ", "Tenth"), inplace=True)
schools_df.rename(columns=lambda s: s.replace("11th ", "Eleventh"), inplace=True)
schools_df.rename(columns=lambda s: s.replace("(", ""), inplace=True)
schools_df.rename(columns=lambda s: s.replace(")", ""), inplace=True)
schools_df.rename(columns=lambda s: s.replace(" ", "_"), inplace=True)

In [ ]:
#@title Drop Tables if They Exist
tables = ['crime', 'census', 'schools']
con = sqlite3.connect('chicago.db')
cur = con.cursor()
listOfTables = cur.execute(
  """SELECT name FROM sqlite_master WHERE type='table'
    AND name = 'crime'; """).fetchall()
sql_string = f"""DROP TABLE crime;"""
if listOfTables != []:
        print(listOfTables)
        print(sql_string)
        cur.execute(sql_string)
listOfTables = cur.execute(
  """SELECT name FROM sqlite_master WHERE type='table'
    AND name = 'census'; """).fetchall()
sql_string = f"""DROP TABLE census;"""
if listOfTables != []:
      print(listOfTables)
      print(sql_string)
      cur.execute(sql_string)
listOfTables = cur.execute(
  """SELECT name FROM sqlite_master WHERE type='table'
    AND name = 'schools'; """).fetchall()
sql_string = f"""DROP TABLE schools;"""
if listOfTables != []:
      print(listOfTables)
      print(sql_string)
      cur.execute(sql_string)

con.close()

[('crime',)]
DROP TABLE crime;
[('census',)]
DROP TABLE census;
[('schools',)]
DROP TABLE schools;


# Step 2: Upload the dataframes to SQLite tables

In [ ]:
pd_to_sqlDB(crime_df,
            table_name='crime',
            db_name='chicago.db')

2022-02-21 22:43:51 INFO: SQL DB chicago.db created
2022-02-21 22:43:51 INFO: SQL Table crime created with 21 columns
2022-02-21 22:43:51 INFO: 533 rows uploaded to crime


In [ ]:
pd_to_sqlDB(census_df,
            table_name='census',
            db_name='chicago.db')

2022-02-21 22:43:55 INFO: SQL DB chicago.db created
2022-02-21 22:43:55 INFO: SQL Table census created with 9 columns
2022-02-21 22:43:55 INFO: 78 rows uploaded to census


In [ ]:
pd_to_sqlDB(schools_df,
            table_name='schools',
            db_name='chicago.db')

2022-02-21 22:43:56 INFO: SQL DB chicago.db created
2022-02-21 22:43:56 INFO: SQL Table schools created with 78 columns
2022-02-21 22:43:57 INFO: 566 rows uploaded to schools


# Step 3: Write the SQL query in a string variable

In [ ]:
crime_query_string = """
    SELECT * 
    FROM crime 
    LIMIT 5
"""
census_query_string = """
    SELECT * 
    FROM census 
    LIMIT 5
"""
schools_query_string = """
    SELECT * 
    FROM schools 
    LIMIT 5
"""

# Step 4: Execute the SQL query

## Sample the crime data

In [ ]:
result_df = sql_query_to_pd(crime_query_string, db_name='chicago.db')
result_df.iloc[:10,:5].head()

,ID,CASE_NUMBER,DATE,BLOCK,IUCR
0,3512276,HK587712,2004-08-28,047XX S KEDZIE AVE,890
1,3406613,HK456306,2004-06-26,009XX N CENTRAL PARK AVE,820
2,8002131,HT233595,2011-04-04,043XX S WABASH AVE,820
3,7903289,HT133522,2010-12-30,083XX S KINGSTON AVE,840
4,10402076,HZ138551,2016-02-02,033XX W 66TH ST,820


## Sample the census data

In [ ]:
result_df = sql_query_to_pd(census_query_string, db_name='chicago.db')
result_df.iloc[:10,:5].head()

,COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PERCENT_OF_HOUSING_CROWDED,PERCENT_HOUSEHOLDS_BELOW_POVERTY,PERCENT_AGED_16__UNEMPLOYED
0,1,Rogers Park,7.7,23.6,8.7
1,2,West Ridge,7.8,17.2,8.8
2,3,Uptown,3.8,24.0,8.9
3,4,Lincoln Square,3.4,10.9,8.2
4,5,North Center,0.3,7.5,5.2


## Sample the school data

In [ ]:
result_df = sql_query_to_pd(schools_query_string, db_name='chicago.db')
result_df.iloc[:10,:5].head()

,School_ID,NAME_OF_SCHOOL,Elementary_Middle_or_High_School,Street_Address,City
0,610038,Abraham Lincoln Elementary School,ES,615 W Kemper Pl,Chicago
1,610281,Adam Clayton Powell Paideia Community Academy ...,ES,7511 S South Shore Dr,Chicago
2,610185,Adlai E Stevenson Elementary School,ES,8010 S Kostner Ave,Chicago
3,609993,Agustin Lara Elementary Academy,ES,4619 S Wolcott Ave,Chicago
4,610513,Air Force Academy High School,HS,3630 S Wells St,Chicago


## Query data from two tables

In [ ]:
# Get the community name from area with most number of crimes
# Name not provided in crime data
crime_community_query = """
SELECT community_area_name, community_area_number
FROM census 
WHERE community_area_number =
  (SELECT community_area_number FROM crime
  WHERE community_area_number >= 1
  GROUP BY community_area_number
  ORDER BY COUNT(*) DESC LIMIT 1)
"""
crime_community_result = sql_query_to_pd(crime_community_query, db_name='chicago.db')
crime_community_result

,COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER
0,Austin,25
